In [ ]:
!pip install lightgbm==3.3.5

import lightgbm as lgb
# Read the data from a CSV file named 'medical_cost.csv' into a DataFrame 'df'.
df = pd.read_csv('dataset.csv')

# Display the first few rows of the DataFrame to provide a data preview.
df.head()

# Define a list of columns to analyze
sample = ['sex', 'children', 'smoker', 'region']

# Iterate through the specified columns
for i, col in enumerate(sample):
    # Group the DataFrame by the current column and calculate the mean of 'charges'
    grouped_data = df[[col, 'charges']].groupby(col).mean()
    
    # Print the mean charges for each group within the current column
    print(grouped_data)
    
    # Print a blank line to separate the output for different columns
    print()
    
# Set the figure size for the subplots
plt.subplots(figsize=(15, 15))

# Define the list of columns to visualize
sample = ['sex', 'children', 'smoker', 'region']

# Loop through the specified columns
for i, col in enumerate(sample):
    # Create subplots in a 3x2 grid
    plt.subplot(3, 2, i + 1)
    
    # Create a countplot for the current column
    sns.countplot(data=df, x=col)
    
# Adjust subplot layout for better presentation
plt.tight_layout()

# Display the subplots
plt.show()


# Set the figure size for the subplots
plt.subplots(figsize=(15, 15))

# Define the list of numeric columns to visualize
sample = ['age', 'bmi', 'charges']

# Loop through the specified columns
for i, col in enumerate(sample):
    # Create subplots in a 3x2 grid
    plt.subplot(3, 2, i + 1)
    
    # Create a distribution plot for the current numeric column
    sns.distplot(df[col])
    
# Adjust subplot layout for better presentation
plt.tight_layout()

# Display the subplots
plt.show()


# Apply the natural logarithm transformation to the 'charges' column
df['charges'] = np.log1p(df['charges'])

# Create a distribution plot for the transformed 'charges' column
sns.distplot(df['charges'])

# Display the distribution plot
plt.show()


# Mapping Categorical to Numerical Values

# Map 'sex' column values ('male' to 0, 'female' to 1)
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

# Map 'smoker' column values ('no' to 0, 'yes' to 1)
df['smoker'] = df['smoker'].map({'no': 0, 'yes': 1})

# Display the DataFrame's first few rows to show the transformations
df.head()


# Perform one-hot encoding on the 'region' column
temp = pd.get_dummies(df['region']).astype('int')

# Concatenate the one-hot encoded columns with the original DataFrame
df = pd.concat([df, temp], axis=1)


# Remove 'Id' and 'region' columns from the DataFrame
df.drop(['region'], inplace=True, axis=1)

# Display the updated DataFrame
print(df.head())

# Define the features
features = df.drop('charges', axis=1)

# Define the target variable as 'charges'
target = df['charges']

# Split the data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(features, target,random_state=2023,test_size=0.25)

# Display the shapes of the training and validation sets
X_train.shape, X_val.shape


# Standardize Features

# Use StandardScaler to scale the training and validation data
scaler = StandardScaler()
#Fit the StandardScaler to the training data
scaler.fit(X_train)
# transform both the training and validation data
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)


# Create a LightGBM dataset for training with features X_train and labels Y_train
train_data = lgb.Dataset(X_train, label=Y_train)

# Create a LightGBM dataset for testing with features X_val and labels Y_val,
# and specify the reference dataset as train_data for consistent evaluation
test_data = lgb.Dataset(X_val, label=Y_val, reference=train_data)

num_round = 100
params = {
    'objective': 'regression',
    'metric': 'l2',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}
bst = lgb.train(params, train_data, num_round, valid_sets=[
                test_data], early_stopping_rounds=10)
bst
